In [1]:
import streamlit

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
gemini_api=os.getenv("gemini_api")
openai_api=os.getenv("openai_api")
os.environ['GOOGLE_API_KEY']=gemini_api
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_openai.chat_models import ChatOpenAI
# llm = ChatOpenAI(model="gpt-4o-mini",api_key=openai_api, temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp",   api_key=gemini_api)

from IPython.display import display, Markdown,Image


In [2]:
import requests

##### INPUTS test cases
ip = "192.168.11.1"

##### Creadientials 
api_key = 
userName = 'moveiton'


##### Web Request maker


#### Hitting the APi
def get_ip_all_details(ip_to_search:str, vt_api_key):
    headers = {
    "accept": "application/json",
    "x-apikey": vt_api_key}
    url = "https://www.virustotal.com/api/v3/ip_addresses/"
    response = requests.get(url+ip_to_search, headers=headers)
    result=eval(response.text)
    # attributes=result['data']['attributes']

    # stats=attributes['last_analysis_stats']
    return response

#### Main

resp=get_ip_all_details("adsfa")
resp.json()

{'error': {'code': 'NotFoundError', 'message': 'Resource not found.'}}

In [3]:
from langgraph.graph import StateGraph, END,START
from langgraph.prebuilt import ToolNode,tool_node, create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool

from pydantic import BaseModel, Field
from typing import TypedDict, List, Dict, Union

In [18]:
@tool
def get_ip_all_details(input_ip:str,vt_api_key):
    """Get details related to provided IP address"""

    headers = {
    "accept": "application/json",
    "x-apikey": vt_api_key}
    url = "https://www.virustotal.com/api/v3/ip_addresses/"

    ip_to_search=input_ip
    response = requests.get(url+ip_to_search, headers=headers)
    if response.status_code==200:
        return response.text
    else:
        return "Provided input is not a correct IP address"
tools=[get_ip_all_details]

system_prompt = f"""
    You are a skilled cybersecurity analyst tasked with evaluating IP addresses. 
    Your role is to use the VirusTotal tool to gather relevant information about the IP address provided. 
    Analyze the VirusTotal results thoroughly and provide:
    1. A concise summary of key findings, including reputation, associated tags, and analysis results.
    2. Clear and actionable recommendations based on the analysis.
    Ensure your analysis is precise and actionable.
"""

memory=MemorySaver()
graph=create_react_agent(model=llm, tools=tools,checkpointer=memory, state_modifier=system_prompt )

In [19]:
thread={"configurable":{"thread_id":"1"}}
user="hi there"
for i in range(3):
    result=graph.invoke({"messages":user},config=thread)
    ai_message=result['messages'][-1].content
    user=input(prompt=ai_message)

In [21]:
result['messages']

[HumanMessage(content='hi there', additional_kwargs={}, response_metadata={}, id='b233369e-1b90-4812-b491-da0edf569cb6'),
 AIMessage(content="Hello! I'm ready to analyze an IP address for you. Please provide the IP address you'd like me to investigate.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'MEDIUM', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-dd4e1dc8-d8d7-4be3-980b-ccc90ccc14b8-0', usage_metadata={'input_tokens': 139, 'output_tokens': 28, 'total_tokens': 167, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='this is the ip address 192.168.1

In [3]:
import requests

def get_ip_location(ip_address):
    
    url = f"https://api.iplocation.net/?ip={ip_address}"

    try:
        
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.text            
            return data
        else:
            return f"Error: Cant retrieve data (Status code: {response.status_code})"
    except Exception as e:
        return f"Error: {str(e)}"

#### MAIN
ip_address = "111.68.97.29"
location_info = get_ip_location(ip_address)
print(location_info)

{"ip":"111.68.97.29","ip_number":"1866752285","ip_version":4,"country_name":"Pakistan","country_code2":"PK","isp":"IP Allocation 1 NUST","response_code":"200","response_message":"OK"}


In [4]:
type(location_info)

str